In [94]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Administrator\.huggingface\token
Login successful


In [95]:
from datasets import load_dataset,Dataset
import pandas as pd

df = pd.read_excel('ptt.xlsx',index_col=0)
df['url'] = df['title'].copy()
df.columns = ['title','summary','text']
df = df[['text','summary','title']]
df = df.dropna(axis=0)
df = Dataset.from_pandas(df)
df = df.remove_columns(column_names=['__index_level_0__'])
df = df.train_test_split(test_size=0.2)
df["train"]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1582
})

In [99]:
from transformers import AutoTokenizer
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom")
model = AutoModel.from_pretrained("bigscience/bloom")

loading file tokenizer.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--bigscience--bloom\snapshots\b366374a32ade3085b3d61dec0cfc424721cdcbd\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--bigscience--bloom\snapshots\b366374a32ade3085b3d61dec0cfc424721cdcbd\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--bigscience--bloom\snapshots\b366374a32ade3085b3d61dec0cfc424721cdcbd\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--bigscience--bloom\snapshots\b366374a32ade3085b3d61dec0cfc424721cdcbd\config.json
Model config BloomConfig {
  "_name_or_path": "bigscience/bloom",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomModel"
  ],
  "attention_dropout": 0.0,


Downloading:   0%|          | 0.00/7.19G [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_df = df.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

NameError: name 'model' is not defined

In [ ]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_new_title_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=tokenized_df["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_new_title_model",
    tokenizer=tokenizer,
)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="skywalker0803r/my_awesome_new_title_model")
summarizer(text)